In [1]:
import datetime
import ipywidgets as widgets
from IPython.display import display

In [2]:
key_dict = {}
keys = []
with open("api_keys.txt", 'r') as fp:
    for i, line in enumerate(fp.readlines()[1:]):
        arr = line.strip().split(',')
        key_dict[i] = ','.join(arr)
        keys.append((arr[0], i))

In [3]:
w_symbol = widgets.Text(
    # value='',
    placeholder='如BTC,后台会自动补齐以满足永续合约格式',
    description='交易币种:',
    disabled=False
)
w_amount = widgets.FloatText(
    # value='',
    placeholder='开仓时必填',
    description='开仓金额/U:',
    disabled=False   
)
w_api_key = widgets.Text(
    # value='8d0cffcb-6765-4566-bd59-72a4b6b26180',
    # placeholder='输入',
    description='API_KEY:',
    disabled=False
)
w_secret = widgets.Text(
    # value='DB0C16E298D8AE45A57C8B20168391F4',
    # placeholder='输入',
    description='SECRET:',
    disabled=False
)
w_passphrase = widgets.Text(
    # value='111@Cmfchina.com',
    # placeholder='输入',
    description='PASSPHRASE:',
    disabled=False
)
w_mode = widgets.Dropdown(
    options=[('实盘', '0'), ('模拟', '1')],
    value='0',
    description='交易模式:',
)
w_key = widgets.Dropdown(
    options=keys,
    value=None,
    description='api_key名称:',
)
def on_key_name_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        key_id = change['new']
        values = key_dict[key_id].split(',')
        w_api_key.value = values[1]
        w_secret.value = values[2]
        w_passphrase.value = values[3]
    
w_key.observe(on_key_name_change)

In [4]:
from okx import getRecent10FundingRatesMax
from okx import getCurrentAccountFundingRatesFlow
from okx import openOrder
from okx import ccyClearance
from okx import doMonitor

# for test
# def getRecent10FundingRatesMax():
#     print('getRecent10FundingRatesMax')
# def getCurrentAccountFundingRatesFlow(*args):
#     print('getCurrentAccountFundingRatesFlow: ', args)
# def openOrder(*args):
#     print('openOrder: ', args)
# def ccyClearance(*args):
#     print('ccyClearance: ', args)
# def doMonitor(*args):
#     print('doMonitor: ', args)

output_log = widgets.Output()

In [5]:
def get_public_params():
    api_key = w_api_key.value
    secret = w_secret.value
    passphrase = w_passphrase.value
    mode = w_mode.value
    return api_key,secret,passphrase,mode

layout_flex = widgets.Layout(width='auto', height='auto') #set width and height
w_btn_get_top_n = widgets.Button(
    description='获取最近十次资金费率最大的十个币种',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_get_top_n(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with output_log:
        getRecent10FundingRatesMax()
w_btn_get_top_n.on_click(click_w_btn_get_top_n)


w_btn_get_account_flow = widgets.Button(
    description='获取当前账户资金费流水',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_get_account_flow(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with output_log:
        (api_key,secret,passphrase,mode) = get_public_params()
        getCurrentAccountFundingRatesFlow(api_key,secret,passphrase,mode)
w_btn_get_account_flow.on_click(click_w_btn_get_account_flow)


w_btn_open_order = widgets.Button(
    description='开仓',
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
)
def click_w_btn_open_order(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    (api_key,secret,passphrase,mode) = get_public_params()
    ccy = w_symbol.value
    amount = w_amount.value
    with output_log:
        print(t,', 帐户: ', api_key, ", 开仓: ", ccy, ', 金额: ', amount)
        openOrder(api_key, secret, passphrase, mode, ccy, amount)
w_btn_open_order.on_click(click_w_btn_open_order)

w_btn_clear_order = widgets.Button(
    description='平仓',
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
)
def click_w_btn_clear_order(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    (api_key,secret,passphrase,mode) = get_public_params()
    ccy = w_symbol.value
    amount = w_amount.value
    with output_log:
        print(t,', 帐户: ', api_key, ", 平仓: ", ccy)
        ccyClearance(api_key, secret, passphrase, mode, ccy)
w_btn_clear_order.on_click(click_w_btn_clear_order)


w_btn_monitor = widgets.Button(
    description='风险监控',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_monitor(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with output_log:
        (api_key,secret,passphrase,mode) = get_public_params()
        doMonitor(api_key,secret,passphrase,mode)
w_btn_monitor.on_click(click_w_btn_monitor)


w_btn_clean = widgets.Button(
    description='清空打印信息',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_clean(b):
    output_log.clear_output()
w_btn_clean.on_click(click_w_btn_clean)

In [8]:
from ipywidgets import HBox, VBox
from ipywidgets import GridspecLayout

grid = GridspecLayout(6, 3)
grid[0,:] = HBox([w_mode, w_key])
grid[1,:] = HBox([w_api_key, w_secret, w_passphrase])
grid[2,:] = HBox([w_symbol, w_amount])
grid[3,:] = HBox([w_btn_open_order, w_btn_clear_order])
grid[4,:] = HBox([w_btn_get_top_n, w_btn_get_account_flow])
# grid[5,:] = HBox([w_btn_monitor, w_btn_clean])
grid[5,:] = HBox([w_btn_clean])
grid

GridspecLayout(children=(HBox(children=(Dropdown(description='交易模式:', options=(('实盘', '0'), ('模拟', '1')), valu…

In [7]:
display(output_log)

Output()